# 11-Genre Deep Models
### **Genre Label: 11**
### **Data: Under sampling (max 7000 for train) || clean_df**
### **Bert Model: Uncase**

In [ ]:
#@title Installs
!pip install pydot --quiet
!pip install gensim --quiet
!pip install tensorflow==2.15.0 --quiet #15 13
!pip install tf_keras==2.15.0 --quiet
!pip install tensorflow-datasets==4.8 --quiet #8
!pip install tensorflow-text==2.15.0 --quiet #15
!pip install transformers==4.17 --quiet #4.40.2 #4.37.2



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorstore 0.1.63 requires ml-dtypes>=0.3.1, but you have ml-dtypes 0.2.0 which is incompatible.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.15.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 73.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import os
import time
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

from tqdm import tqdm
from multiprocessing import Pool, cpu_count


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Pre-Processing

In [ ]:
cleaned_df = pd.read_pickle('/content/drive/MyDrive/w266/266 Final Project/Music4all Dataset/cleaned_df_2.pkl')

In [ ]:
train_df, remaining_df = train_test_split(cleaned_df, test_size=0.20, random_state=42)
val_df, test_df = train_test_split(remaining_df, test_size=0.50, random_state=42)

print('Training set shape:', train_df.shape)
print('Validation set shape:', val_df.shape)
print('Test set shape:', test_df.shape)

Training set shape: (59256, 8)
Validation set shape: (7407, 8)
Test set shape: (7407, 8)


In [ ]:
# random undersample to fit the lowest class which is alternative
from sklearn.utils import resample

max_samples = 7000
undersampled_train_df = pd.DataFrame()

for genre in train_df['broad_genre'].unique():
    genre_df = train_df[train_df['broad_genre'] == genre]
    if len(genre_df) > max_samples:
        genre_df = resample(genre_df, replace=False, n_samples=max_samples, random_state=42)
    undersampled_train_df = pd.concat([undersampled_train_df, genre_df])

print('Training set genre distribution:')
print(undersampled_train_df['broad_genre'].value_counts())
train_df = undersampled_train_df

Training set genre distribution:
broad_genre
Pop               7000
Rock              7000
Rhythm & Blues    5505
Country           5058
Electronic        4324
Metal             4092
Punk              3927
Jazz              3469
Folk              3279
Hip Hop & Rap     2693
Reggae            2060
Name: count, dtype: int64


In [ ]:
# getting the text and tables from train, validation and test
train_text = train_df['lyrics'].tolist()
val_text = val_df['lyrics'].tolist()
test_text = test_df['lyrics'].tolist()

train_label = train_df['broad_genre'].tolist()
val_label = val_df['broad_genre'].tolist()
test_label = test_df['broad_genre'].tolist()

In [ ]:
train_df['lyric_length'] = train_df.lyrics.apply(lambda x: len(x.split()))
val_df['lyric_length'] = val_df.lyrics.apply(lambda x: len(x.split()))
test_df['lyric_length'] = test_df.lyrics.apply(lambda x: len(x.split()))

In [ ]:
numerical_features = ['release', 'danceability', 'energy', 'valence', 'lyric_length']

# scaling numerical features
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(train_df[numerical_features])
X_val_scale = scaler.transform(val_df[numerical_features])
X_test_scale = scaler.transform(test_df[numerical_features])

In [ ]:
# getting rid of \n in the lyrics column
def preprocess_text(text):
  return text.replace('\n', ' ')

train_text = [preprocess_text(text) for text in train_text]
val_text = [preprocess_text(text) for text in val_text]
test_text = [preprocess_text(text) for text in test_text]


In [ ]:
encode_label = LabelEncoder()
train_label = encode_label.fit_transform(train_label)
val_label = encode_label.transform(val_label)
test_label = encode_label.transform(test_label)

### Tokenize dataset with expanded genre label

In [ ]:
word_len = [len(w.split()) for w in train_text]

sum(word_len) / len(word_len)

188.60544962505423

In [ ]:
MAX_SEQUENCE_LENGTH = 250

In [ ]:
from transformers import BertTokenizer, TFBertModel
import torch

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# training
train_tokenized = bert_tokenizer(train_text,
                                 max_length=MAX_SEQUENCE_LENGTH,
                                 truncation=True,
                                 padding='max_length',
                                 return_tensors='tf')
train_inputs = [train_tokenized.input_ids, train_tokenized.token_type_ids, train_tokenized.attention_mask]
train_inputs_num = [train_tokenized.input_ids,
                train_tokenized.token_type_ids,
                train_tokenized.attention_mask,
                X_train_scale]
train_labels = np.array(train_label)

# validation
val_tokenized = bert_tokenizer(val_text,
                               max_length=MAX_SEQUENCE_LENGTH,
                               truncation=True,
                               padding='max_length',
                               return_tensors='tf')
val_inputs = [val_tokenized.input_ids, val_tokenized.token_type_ids, val_tokenized.attention_mask]
val_inputs_num = [val_tokenized.input_ids,
              val_tokenized.token_type_ids,
              val_tokenized.attention_mask,
              X_val_scale]
val_labels = np.array(val_label)

# test
test_tokenized = bert_tokenizer(test_text,
                                max_length=MAX_SEQUENCE_LENGTH,
                                truncation=True,
                                padding='max_length',
                                return_tensors='tf')
test_inputs = [test_tokenized.input_ids, test_tokenized.token_type_ids, test_tokenized.attention_mask]
test_inputs_num = [test_tokenized.input_ids,
               test_tokenized.token_type_ids,
               test_tokenized.attention_mask,
               X_test_scale]
test_labels = np.array(test_label)

In [ ]:
print('Train input IDs shape:', train_inputs[0].shape)
print('Validation input IDs shape:', val_inputs[0].shape)
print('Test input IDs shape:', test_inputs[0].shape)
print('First tokenized train input:', train_inputs[0][0])
print('First tokenized validation input:', val_inputs[0][0])
print('First tokenized test input:', test_inputs[0][0])

Train input IDs shape: (48407, 250)
Validation input IDs shape: (7407, 250)
Test input IDs shape: (7407, 250)
First tokenized train input: tf.Tensor(
[  101  2514  4558  7200  2113  2272  8307  2963  3275  4558  2617 10436
  4452  3959  2895  6069 12985  2364  8432  5913  3342  2735  2157  3426
  2113  2444  9647  4826  7955 18987  3377  3342 12342  3362  2152  2514
  2907  2292  2066 12342 12342  2051  2088  2210  3553  2146  3201  2157
  2146  2514  2503  2113  4452  3959  2895  6069 12985  2364  8432  5913
  3342  2735  2157  3426  2113  2444  9647  4826  7955 18987  3377  3342
 12342  2425  2360  6211  2051  2689  2568 10587  2215  2342  2066  2342
  7200  4797  2903  4452  3959  2895  6069 12985  2364  8432  5913  3342
  2735  2157  3426  2113  2444  9647  4826  7955 18987  3377  3342 12342
   102     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     

## Model function

In [ ]:
#@title Text-Only BERT Model

def text_only_bert(bert_base_model,
                    max_sequence_length=MAX_SEQUENCE_LENGTH,
                    hidden_size = 100,
                    num_hidden = 1,
                    dropout=0.3,
                    learning_rate=2e-5,
                    last_layer_num=11,
                    num_classes=14
                  ):
    # freezing all bert layers except the last transformer block
    for w in bert_base_model.weights:
        if f'layer_._{last_layer_num}' not in w.name and 'pooler' not in w.name:
            w._trainable = False

    input_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}

    bert_out = bert_base_model(bert_inputs)

    cls_token = bert_out[0][:, 0, :]

    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(cls_token)
    hidden = tf.keras.layers.Dropout(dropout)(hidden)

    for i in range(num_hidden - 1):
      hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name=f'hidden_layer_{i+1}')(hidden)
      hidden = tf.keras.layers.Dropout(dropout)(hidden)


    classification = tf.keras.layers.Dense(num_classes, activation='softmax', name='classification_layer')(hidden)
    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0),
                                 loss='sparse_categorical_crossentropy',
                                 metrics=['accuracy'])
    return classification_model

In [ ]:
#@title Text & Numerical Feature BERT Model

def text_numerical_bert(bert_base_model,
                          max_sequence_length=MAX_SEQUENCE_LENGTH,
                          hidden_size = 100,
                          num_hidden = 1,
                          dropout=0.3,
                          learning_rate=2e-5,
                          last_layer_num=11,
                          num_classes=14,
                          num_non_text_features=5
                       ):
    # freezing all bert layers except the last transformer block
    for w in bert_base_model.weights:
        if f'layer_._{last_layer_num}' not in w.name and 'pooler' not in w.name:
            w._trainable = False

    input_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}

    bert_out = bert_base_model(bert_inputs)

    cls_token = bert_out[0][:, 0, :]

    # Non-text features input
    non_text_features_input = tf.keras.layers.Input(shape=(num_non_text_features,), dtype=tf.float32, name='non_text_features')

    # Combine CLS token with non-text features
    combined_features = tf.keras.layers.Concatenate(name='combined_features')([cls_token, non_text_features_input])


    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(combined_features)
    hidden = tf.keras.layers.Dropout(dropout)(hidden)

    for i in range(num_hidden - 1):
      hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name=f'hidden_layer_{i+1}')(hidden)
      hidden = tf.keras.layers.Dropout(dropout)(hidden)


    classification = tf.keras.layers.Dense(num_classes, activation='softmax', name='classification_layer')(hidden)
    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask, non_text_features_input], outputs=[classification])
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0),
                                 loss='sparse_categorical_crossentropy',
                                 metrics=['accuracy'])
    return classification_model

## Train Models

In [ ]:
num_classes = len(encode_label.classes_)
checkpoint_dir = '/content/drive/MyDrive/w266/266 Final Project/model_checkpoints/'
bert_last_layer_num = 11

### Text-Only BERT Model

In [ ]:
bert_model_text = text_only_bert(
    bert_model,
    max_sequence_length=MAX_SEQUENCE_LENGTH,
    hidden_size=200,
    dropout=0.3,
    learning_rate=2e-5,
    last_layer_num=bert_last_layer_num,
    num_classes = num_classes
)

bert_model_text.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 attention_mask_layer (Inpu  [(None, 250)]                0         []                            
 tLayer)                                                                                          
                                                                                                  
 input_ids_layer (InputLaye  [(None, 250)]                0         []                            
 r)                                                                                               
                                                                                                  
 token_type_ids_layer (Inpu  [(None, 250)]                0         []                            
 tLayer)                                                                                      

In [ ]:
#calculate sample weights
sample_weight = np.ones(shape=(len(train_labels),))
for label in range(np.max(train_labels) + 1):
    class_weight = 1 - np.sum(train_labels == label) / len(train_labels)
    sample_weight[train_labels == label] = class_weight

In [ ]:
bert_model_text_history = bert_model_text.fit(
    train_inputs,
    train_labels,
    sample_weight=sample_weight,
    validation_data=(val_inputs, val_labels),
    epochs=6,
    batch_size=8
)


Epoch 1/6


6051/6051 [==============================] - 1244s 204ms/step - loss: 1.5734 - accuracy: 0.3471 - val_loss: 1.5844 - val_accuracy: 0.4029
Epoch 2/6
6051/6051 [==============================] - 1230s 203ms/step - loss: 1.4353 - accuracy: 0.4059 - val_loss: 1.5503 - val_accuracy: 0.4092
Epoch 3/6
6051/6051 [==============================] - 1229s 203ms/step - loss: 1.3811 - accuracy: 0.4315 - val_loss: 1.5281 - val_accuracy: 0.4332
Epoch 4/6
6051/6051 [==============================] - 1229s 203ms/step - loss: 1.3390 - accuracy: 0.4500 - val_loss: 1.5130 - val_accuracy: 0.4409
Epoch 5/6
6051/6051 [==============================] - 1228s 203ms/step - loss: 1.3024 - accuracy: 0.4651 - val_loss: 1.5029 - val_accuracy: 0.4377
Epoch 6/6
6051/6051 [==============================] - 1228s 203ms/step - loss: 1.2710 - accuracy: 0.4813 - val_loss: 1.5199 - val_accuracy: 0.4415


In [ ]:
val_predictions = bert_model_text.predict(val_inputs)
val_predictions = np.argmax(val_predictions, axis=1)
print(classification_report(val_labels, val_predictions, target_names=encode_label.classes_))


232/232 [==============================] - 129s 545ms/step
                precision    recall  f1-score   support

       Country       0.44      0.67      0.53       600
    Electronic       0.29      0.28      0.29       509
          Folk       0.36      0.34      0.35       402
 Hip Hop & Rap       0.82      0.75      0.78       343
          Jazz       0.36      0.46      0.40       462
         Metal       0.57      0.70      0.62       526
           Pop       0.48      0.42      0.45      1489
          Punk       0.31      0.44      0.36       463
        Reggae       0.51      0.40      0.45       290
Rhythm & Blues       0.32      0.38      0.35       693
          Rock       0.51      0.33      0.40      1630

      accuracy                           0.44      7407
     macro avg       0.45      0.47      0.45      7407
  weighted avg       0.45      0.44      0.44      7407



### Text & Numerical Feature BERT Model

In [ ]:
bert_model_num = text_numerical_bert(
    bert_model,
    max_sequence_length=MAX_SEQUENCE_LENGTH,
    hidden_size=200,
    dropout=0.3,
    learning_rate=2e-5,
    last_layer_num=bert_last_layer_num,
    num_classes = num_classes
)

bert_model_num.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 attention_mask_layer (Inpu  [(None, 250)]                0         []                            
 tLayer)                                                                                          
                                                                                                  
 input_ids_layer (InputLaye  [(None, 250)]                0         []                            
 r)                                                                                               
                                                                                                  
 token_type_ids_layer (Inpu  [(None, 250)]                0         []                            
 tLayer)                                                                                    

In [ ]:
bert_model_num_history = bert_model_num.fit(
    train_inputs_num,
    train_labels,
    sample_weight=sample_weight,
    validation_data=(val_inputs_num, val_labels),
    epochs=6,
    batch_size=8
)


Epoch 1/6


6051/6051 [==============================] - 1241s 204ms/step - loss: 1.3999 - accuracy: 0.4339 - val_loss: 1.5218 - val_accuracy: 0.4403
Epoch 2/6
6051/6051 [==============================] - 1230s 203ms/step - loss: 1.2647 - accuracy: 0.4868 - val_loss: 1.4490 - val_accuracy: 0.4651
Epoch 3/6
6051/6051 [==============================] - 1230s 203ms/step - loss: 1.2014 - accuracy: 0.5144 - val_loss: 1.4412 - val_accuracy: 0.4723
Epoch 4/6
6051/6051 [==============================] - 1228s 203ms/step - loss: 1.1475 - accuracy: 0.5370 - val_loss: 1.4729 - val_accuracy: 0.4655
Epoch 5/6
6051/6051 [==============================] - 1229s 203ms/step - loss: 1.1022 - accuracy: 0.5538 - val_loss: 1.4492 - val_accuracy: 0.4737
Epoch 6/6
6051/6051 [==============================] - 1231s 203ms/step - loss: 1.0649 - accuracy: 0.5695 - val_loss: 1.4520 - val_accuracy: 0.4778


In [ ]:
val_predictions = bert_model_num.predict(val_inputs_num)
val_predictions = np.argmax(val_predictions, axis=1)
print(classification_report(val_labels, val_predictions, target_names=encode_label.classes_))


## Test Models

### Text-Only BERT Model

In [ ]:
test_predictions = bert_model_text.predict(test_inputs)
test_predictions = np.argmax(test_predictions, axis=1)
print(classification_report(test_labels, test_predictions, target_names=encode_label.classes_))

232/232 [==============================] - 126s 545ms/step
                precision    recall  f1-score   support

       Country       0.43      0.69      0.53       624
    Electronic       0.33      0.28      0.30       573
          Folk       0.34      0.33      0.33       401
 Hip Hop & Rap       0.78      0.78      0.78       335
          Jazz       0.30      0.41      0.35       421
         Metal       0.55      0.71      0.62       527
           Pop       0.51      0.43      0.47      1536
          Punk       0.35      0.46      0.40       524
        Reggae       0.49      0.38      0.43       259
Rhythm & Blues       0.31      0.35      0.33       666
          Rock       0.46      0.31      0.37      1541

      accuracy                           0.44      7407
     macro avg       0.44      0.47      0.45      7407
  weighted avg       0.44      0.44      0.43      7407



### Text & Numerical Feature BERT Model

In [ ]:
test_predictions = bert_model_num.predict(test_inputs_num)
test_predictions = np.argmax(test_predictions, axis=1)
print(classification_report(test_labels, test_predictions, target_names=encode_label.classes_))

232/232 [==============================] - 129s 545ms/step
                precision    recall  f1-score   support

       Country       0.46      0.71      0.56       624
    Electronic       0.35      0.41      0.38       573
          Folk       0.50      0.46      0.48       401
 Hip Hop & Rap       0.72      0.81      0.76       335
          Jazz       0.43      0.30      0.35       421
         Metal       0.64      0.68      0.66       527
           Pop       0.53      0.39      0.45      1536
          Punk       0.41      0.48      0.44       524
        Reggae       0.46      0.51      0.48       259
Rhythm & Blues       0.32      0.41      0.36       666
          Rock       0.49      0.42      0.45      1541

      accuracy                           0.47      7407
     macro avg       0.48      0.51      0.49      7407
  weighted avg       0.48      0.47      0.47      7407

